In [2]:
import os
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 设置路径和参数
train_path = r'E:\HKULearning\2025 spring\STAT8021\group work\Krapivin\krapivin-2009-pre-master\src\all_docs_abstacts_refined'
K = 5  # 提取的关键词数量

# 读取训练数据和真实关键词
corpus = []
true_keywords_list = []

for txt_file in glob.glob(os.path.join(train_path, '*.txt')):
    with open(txt_file, 'r', encoding='utf-8') as f:
        corpus.append(f.read())
    
    base_name = os.path.splitext(os.path.basename(txt_file))[0]
    key_file = os.path.join(train_path, f'{base_name}.key')
    keywords = []
    if os.path.exists(key_file):
        with open(key_file, 'r', encoding='utf-8') as kf:
            for line in kf:
                keywords.extend(line.strip().lower().split())
    true_keywords_list.append(set(keywords))

# 训练TF-IDF模型
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()

# 提取Top K关键词
predicted_keywords_list = []
for i in range(tfidf_matrix.shape[0]):
    vector = tfidf_matrix[i].toarray().flatten()
    top_k_indices = np.argsort(vector)[-K:][::-1]
    top_k_words = [feature_names[idx] for idx in top_k_indices if vector[idx] > 0]
    predicted_keywords_list.append(set(top_k_words))

# 计算评估指标
precisions, recalls, f1_scores = [], [], []
for true_set, pred_set in zip(true_keywords_list, predicted_keywords_list):
    tp = len(true_set & pred_set)
    # Precision@K
    precision = tp / K
    precisions.append(precision)
    
    # Recall
    true_len = len(true_set)
    if true_len == 0:
        recall = 0.0  # 若真实关键词为空则跳过
    else:
        recall = tp / true_len
    recalls.append(recall)
    
    # F1 Score
    if (precision + recall) == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    f1_scores.append(f1)

# 输出结果
print(f"[评估结果]")
print(f"Average Precision@{K}: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")
print(f"Average Cosine Similarity: {np.mean(cosine_similarities):.4f}")  # 原有余弦相似度计算保留

[评估结果]
Average Precision@5: 0.1924
Average Recall: 0.1082
Average F1 Score: 0.1330
Average Cosine Similarity: 0.1410
